In [ ]:
from pathlib import Path

import numpy as np

from ipyfilechooser import FileChooser

from astropy.nddata import CCDData
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.table import Table
from astropy.time import Time

from stellarphot.photometry import *
from stellarphot.source_detection import *
from stellarphot import Camera

from stellarphot.visualization.photometry_widget_functions import PhotometrySettings

## Settings are below 

In [ ]:
ps = PhotometrySettings()
ps.box

In [ ]:
# A single dot means "the folder this notebook is in"
folder_with_images = ps.image_folder # '.'

# The file name below should be the one you made with the comp-star template
# Its name should end .fits
source_file_name = ps.aperture_locations # 'aperture_locations.fits'

# Enter the aperture radius from the viewer seeing template here
aperture = ps.aperture_radius

# Enter the object name here
object_name = ps.object_name

# Enter date data was taken here
date = '2021-09-28'

# Enter the name you want to give the photometry file, i.e. the file with star counts, here
photometry_file = f'{object_name}-{date}.csv'

### More settings (no need to change these typically)

In [ ]:
inner_annulus = aperture + 15
outer_annulus = inner_annulus + 15

feder_cg_16m = Camera(gain=1.5, read_noise=10.0, dark_current=0.01)

max_adu = 50000

In [ ]:
image_directory = Path(folder_with_images)

In [ ]:
sources = Table.read(source_file_name)

In [ ]:
if not isinstance(sources['coord'][0], SkyCoord):
    sources['coord'] = SkyCoord(ra=[c.split(',')[0] for c in sources['coord']],
                                dec=[c.split(',')[1] for c in sources['coord']],
                                unit='degree',
                                frame='icrs')
star_ra = sources['coord'].ra
star_dec = sources['coord'].dec

main_target_coords = sources[sources['star_id'] == 1]['coord']

In [ ]:
photometry_table = photometry_on_directory(
                        image_directory, object_name, 
                        (star_ra, star_dec), 
                        aperture, inner_annulus, outer_annulus, 
                        max_adu, sources['star_id'], 
                        feder_cg_16m,
                        observatory_location='feder',
                        bjd_coords=main_target_coords,
                        fwhm_by_fit=False                
                    )

In [ ]:
photometry_table.write(photometry_file)